## McMaster University Comp Sci 4TB3/6TB3, Winter Term 2016/17 â Lab 10
#### For the Labs on March 21 - March 23 <br>Due Friday, March 24, 11 pm<br>Emil Sekerinski, Shucai Yao, Spencer Park<br>out of 24 points; extra points count towards any other lab

To be submitted in the folder Lab10 on GitLab; you should have received an invitation to join the project cs4tb3-winter17/yourmacid.
* You need to download the P0 jupyter notebook posted together with the lab, not use a previous version. You need to modify several notebooks, you should not create any other files.
* This assignment requires a jupyter installation. You can install it on your computer, use UTS labs, or use https://notebooks.azure.com/.
* In this lab, you are allowed to work in pairs, provided that you split the work equally and arrive at a common understanding of the solution. However, in that case you must state in your submission the person you worked with, such that similarities in the solution will not be construed as Academic Dishonesty. Working in groups of three or larger is not allowed and will be considered Academic Dishonesty. If you look for someone to work with, we will try to find a match, please contact the TAs.
* You are allowed and encouraged to talk to everyone in the course to get a common understanding of the problem, but you can share partial solutions only with your collaborator, if you work in a pair. The final submission must be your own, that is, you cannot submit identical submissions with two names on them.
* In the lab sessions, the solution to last week's lab questions are discussed and you can get help with this week's lab questions. Attendance at the labs is not checked.

### Parameter Passing in Registers
The P0 language supports value and reference parameters. The P0 compiler passes all parameters on the stack. On the MIPS processor, by convention, registers `$a0` to `$a3` are used for the first four parameters, provided they fit in a word, and only any further parameters are passed on the stack. Consider:

In [1]:
import nbimporter
from P0 import compileString
from ST import printSymTab

compileString("""
program parameterpassing;
  var a: boolean;
  var i, j: integer;
  procedure p(var u: boolean; v: boolean; w: integer; var x, y: integer; z: integer);
    begin  
      u := true; v := u and v; x := w + 3; w := 5; y := 7 - z
    end;
  begin p(a, true, 7, i, j, 9);
    if a then write(j)
  end
""")

importing Jupyter notebook from P0.ipynb
importing Jupyter notebook from SC.ipynb


importing Jupyter notebook from ST.ipynb


importing Jupyter notebook from CGmips.ipynb
	.data
j_:	.space 4
i_:	.space 4
a_:	.space 4
	.text
	.globl p
	.ent p
p:	
	sw $fp, -12($sp)
	sw $ra, -16($sp)
	sub $fp, $sp, 8
	sub $sp, $fp, 8
	addi $t6, $0, 1
	sw $t6, 0($a0)
	lw $t3, 0($a0)
	beq $t3, $0, L1
L2:	
	add $t5, $a1, $0
	beq $t5, $0, L3
L4:	
	addi $a1, $0, 1
	b, L5
L3:	
L1:	
	addi $a1, $0, 0
L5:	
	add $t2, $a2, 3
	sw $t2, 0($a3)
	addi $a2, $0, 5
	lw $t7, 4($fp)
	addi $t0, $0, 7
	lw $t8, 0($fp)
	sub $t0, $t0, $t8
	sw $t0, 0($t7)
	add $sp, $fp, 8
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.text
	.globl main
	.ent main
main:	
	la $a0, a_
	addi $a1, $0, 1
	addi $a2, $0, 7
	la $a3, i_
	la $t4, j_
	sw $t4, -4($sp)
	addi $t1, $0, 9
	sw $t1, -8($sp)
	jal, p
	lw $t6, a_
	beq $t6, $0, L6
L7:	
	lw $a0, j_
	li $v0, 1
	syscall
L6:	
	li $v0, 10
	syscall
	.end main


### Lab Question 1 (Annotating Generated Code) [8 points]
For above program, P0 generates:
```


        .data
j_:     .space 4
i_:     .space 4
a_:     .space 4
        .text
        .globl p
        .ent p
p:      
                          # offset(u) = 20
                          # offset(v) = 16
                          # offset(w) = 12
                          # offset(x) = 8
                          # offset(y) = 4
                          # offset(z) = 0
        sw $fp, -28($sp)  # M[$sp - parametersize - 4] = $fp
        sw $ra, -32($sp)  # M[parametersize + 8 = 32
        sub $fp, $sp, 24  # $fp := $sp - parametersize
        sub $sp, $fp, 8   # $sp := $fp - 8
        lw $t0, 20($fp)   # $t0 := M[$fp + offset(u)]   [ = adr(u)]
        addi $t5, $0, 1   # $t5 := 1                    [ = true]
        sw $t5, 0($t0)    # [M[adr(u) = ] M[$t0] := $t5 (u = true)
        lw $t2, 20($fp)   # $t2 := M[$fp + offset(u)]   [ = adr(u)]
        lw $t6, 0($t2)    # $t6 := M[$t2]               [ = u]
        beq $t6, $0, L1   # if u == 0 goto L1
L2:     
        lw $t8, 16($fp)   # $t8 := M[$fp + offset(v)]   [ = v]
        beq $t8, $0, L3   # if v == 0 goto L3
L4:     
        addi $t4, $0, 1   # $t4 := 1                    [ = true]
        b, L5             # goto L5
L3:     
L1:     
        addi $t4, $0, 0   # $t4 := 0                    [= false]
L5:     
        sw $t4, 16($fp)   # M[$fp + offset(v)] := $t4   [v = false]
        lw $t1, 8($fp)    # $t1 = M[$fp + offset(x)]    [= adr(x)]
        lw $t7, 12($fp)   # $t7 = M[$fp + offset(w)]    [= w]
        add $t7, $t7, 3   # $t7 := $t7 + 3              [w += 3]      
        sw $t7, 0($t1)    # M[adr(x)] := $t7            [x = w+3]
        addi $t3, $0, 5   # $t3 := 5
        sw $t3, 12($fp)   # M[$fp + offset(w)] := $t3   [w = 5]
        lw $t5, 4($fp)    # $t5 := M[$fp + offset(y)]   [= adr(y)]
        addi $t2, $0, 7   # $t2 := 7
        lw $t6, 0($fp)    # $t6 := M[$fp + offset(z)]   [= z]
        sub $t2, $t2, $t6 # $t2 := 7 - z
        sw $t2, 0($t5)    # $M[adr(y)] := 7 - z         [y = 7 - z]
        add $sp, $fp, 24  # $sp := $fp + parametersize  [pop params]
        lw $ra, -8($fp)   # $ra := M[$fp - 8]           [load last $ra]
        lw $fp, -4($fp)   # $fp := M[$fp - 4]           [load last $fp]
        jr $ra
        .text
        .globl main
        .ent main
main:   
        la $t8, a_        # $t8 := adr(a)
        sw $t8, -4($sp)   # M[$sp - 4] := $t8
        addi $t4, $0, 1   # $t4 := 1     [=true]
        sw $t4, -8($sp)   # M[$sp - 8] := 1
        addi $t7, $0, 7   # $t7 = 7
        sw $t7, -12($sp)  # M[$sp - 12] := 7
        la $t3, i_        # $t3 := adr(i)
        sw $t3, -16($sp)  # M[$sp - 16] := adr(i)
        la $t2, j_        # $t2 := adr(j)
        sw $t2, -20($sp)  # M[$sp -20] := adr(j)
        addi $t6, $0, 9   # $t6 := 9
        sw $t6, -24($sp)  # M[$sp -24] := 9
        jal, p            # call p
        lw $t8, a_        # $t8 = a
        beq $t8, $0, L6   # if a == 0 goto L6
L7:     
        lw $a0, j_        # $a0 = j
        li $v0, 1         # $v0 = 1  [op code to write]
        syscall
L6:     
        li $v0, 10        # $v0 = 10 [op code to exit]
        syscall
        .end main
```
Add the missing annotation.

### Lab Question 2 (Parameters in Registers) [16 points]
Modify the `CGmips` to use `$a0` to `$a3` for the first four parameters. The code for above example should be:
```
        .data
j_:     .space 4
i_:     .space 4
a_:     .space 4
        .text
        .globl p
        .ent p
p:      
        sw $fp, -12($sp)
        sw $ra, -16($sp)
        sub $fp, $sp, 8
        sub $sp, $fp, 8
        addi $t0, $0, 1
        sw $t0, 0($a0)
        lw $t7, 0($a0)
        beq $t7, $0, L1
L2:     
        add $t4, $a1, $0
        beq $t4, $0, L3
L4:     
        addi $a1, $0, 1
        b, L5
L3:     
L1:     
        addi $a1, $0, 0
L5:     
        add $t2, $a2, 3
        sw $t2, 0($a3)
        addi $a2, $0, 5
        lw $t6, 4($fp)
        addi $t8, $0, 7
        lw $t1, 0($fp)
        sub $t8, $t8, $t1
        sw $t8, 0($t6)
        add $sp, $fp, 8
        lw $ra, -8($fp)
        lw $fp, -4($fp)
        jr $ra
        .globl main
        .ent main
main:   
        la $a0, a_
        addi $a1, $0, 1
        addi $a2, $0, 7
        la $a3, i_
        la $t3, j_
        sw $t3, -4($sp)
        addi $t5, $0, 9
        sw $t5, -8($sp)
        jal, p
        lw $t0, a_
        beq $t0, $0, L6
L7:     
        lw $a0, j_
        li $v0, 1
        syscall
L6:     
        li $v0, 10
        syscall
        .end main
```
Test the generated MIPS code by running it in SPIM, http://spimsimulator.sourceforge.net/. Consider following procedures:
* `genVar(x)` takes the symbol table entry of variable `x` and returns a `Var` item with its location, which is either `$fp` relative for parameters and local variables or an absolute address for global variables. With parameter passing in registers, a variable can also be in one of `$a0`, ..., `$a3`. Function `genVar(x)` has already been modified accordingly.
* `genAssign(x, y)` generates code for `x := y`, where `x` is a `Var` item. Now `x` can be a `Reg` item: if `y` is a `Cond`, then `1` (for `true`) or `0` (for `false`) can be loaded directly in the register of `x` (rather than in a newly obtained register that is stored at the address of `x`); similarly, if `y` is a memory location, that can be loaded directly in the register of `x`; if `y` is a `Reg` as well, then `y` needs to be moved `x`.
* `genFormalParams(sc)`: instead of assigning `$fp` and the offset on the stack to the `reg` and `adr` fields of all parameters, the first four parameters need to be assigned to registers `$a0`, ..., `$a3`.
* `genActualPara(ap, fp, n)`: instead of "pushing" all parameters on the stack, for the first four parameters, registers `$a0`, ..., `$a3` are used. There are three cases: the parameter a reference parameter and is already in a register, in which case the value needs to be moved to one of `$a0`, ..., `$a3`; the parameter is a reference parameter and needs to be loaded in a register, in which case one of `$a0`, ..., `$a3` is used; or the parameter is a value parameter, in which case the parameter needs to be loaded in one of `$a0`, ..., `$a3`.

### Lab Question 3 (Fixing Procedure Calls) [12 points]
Above scheme will generate _incorrect_ code when a procedure calls a procedure:

In [2]:
compileString("""
program procedurecallingprocedure;
  procedure p(x, y: integer);
    begin write(x); write(y)
    end;
  procedure q(a, b: integer);
  begin p(3, 5); write(a); write(b)
  end;
  begin q(7, 9)
  end
""", 'P1.s')

The generated code is:
```
        .data
        .text
        .globl p
        .ent p
p:      
        sw $fp, -4($sp)
        sw $ra, -8($sp)
        sub $fp, $sp, 0
        sub $sp, $fp, 8
        add $a0, $a0, $0
        li $v0, 1
        syscall
        add $a0, $a1, $0
        li $v0, 1
        syscall
        add $sp, $fp, 0
        lw $ra, -8($fp)
        lw $fp, -4($fp)
        jr $ra
        .globl q
        .ent q
q:      
        sw $fp, -4($sp)
        sw $ra, -8($sp)
        sub $fp, $sp, 0
        sub $sp, $fp, 8
        addi $a0, $0, 3
        addi $a1, $0, 5
        jal, p
        add $a0, $a0, $0
        li $v0, 1
        syscall
        add $a0, $a1, $0
        li $v0, 1
        syscall
        add $sp, $fp, 0
        lw $ra, -8($fp)
        lw $fp, -4($fp)
        jr $ra
        .globl main
        .ent main
main:   
        addi $a0, $0, 7
        addi $a1, $0, 9
        jal, q
        li $v0, 10
        syscall
        .end main
```
The call of `p` will reuse registers `$a0` and `$a1` without saving the previous values. Implement a scheme that fixes this problem. One way to fix this is to save any argument registers that are in use before the next call.